In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
from typing import Tuple, List
import time

#Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
#!pip install einops
#!pip install bitsandbytes
#!pip install transformers --upgrade
#!pip install torch --upgrade
#!pip install --upgrade accelerate

In [3]:
import einops
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
#from .autonotebook import tqdm as notebook_tqdm

C:\Users\riede\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import sys
print("Python version")
print (sys.version)
print(transformers.__version__)


print(torch.__version__)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('using device:', device)

# Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3, 1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3, 1), 'GB')


Python version
3.11.0 (main, Oct 24 2022, 18:26:48) [MSC v.1933 64 bit (AMD64)]
4.30.2
2.0.1+cu118
using device: cuda
NVIDIA GeForce GTX 970
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [6]:
def measure_perf(
    prompt: str, model: AutoModelForCausalLM, tokenizer: AutoTokenizer
) -> Tuple[float, float, torch.Tensor]:
    """
    Measures memory consumption and inference execution time for a given model and prompt.

    Args:
        prompt: Text to be used as input for the model.
        model: Pretrained model used for inference.
        tokenizer: Pretrained tokenizer used to encode the prompt.

    Returns:
        Peak memory consumption in GB, execution time in seconds, and output tensor from the model.
    """
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()

    start_time = time.time()

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
    outputs = model.generate(input_ids, max_length=100)

    end_time = time.time()

    peak_mem = torch.cuda.max_memory_allocated()
    peak_mem_consumption = peak_mem / 1e9  # convert bytes to GB

    exec_time = end_time - start_time

    return peak_mem_consumption, exec_time, outputs

In [7]:
def plot_results(
    mem_consumptions: List[float], execution_times: List[float], dir: str = ""
) -> None:
    """
    Plots memory consumption and execution times.

    Args:
        mem_consumptions: List of memory consumption data in GB.
        execution_times: List of execution time data.
        dir: Destination dir for the plot.
    """
    #os.makedirs(dir, exist_ok=True)

    fig, ax1 = plt.subplots()

    color = "tab:red"
    ax1.set_xlabel("Runs")
    ax1.set_ylabel("GPU Memory Consumption (GB)", color=color)
    ax1.plot(mem_consumptions, color=color)
    ax1.tick_params(axis="y", labelcolor=color)
    ax1.yaxis.get_major_formatter().set_useOffset(False)

    ax2 = ax1.twinx()
    color = "tab:blue"
    ax2.set_ylabel("Execution time (s)", color=color)
    ax2.plot(execution_times, color=color)
    ax2.tick_params(axis="y", labelcolor=color)
    ax2.yaxis.get_major_formatter().set_useOffset(False)

    fig.tight_layout()
    plt.title("GPU Memory Consumption and Execution Time for Each Run")
    fig.subplots_adjust(top=0.88)
    plt.savefig(f"{dir}/falcon_memory_time.png")

In [8]:
model_path = "tiiuae/falcon-7b-instruct"
config = AutoConfig.from_pretrained(model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    config=config,
    trust_remote_code=True,
    load_in_4bit=True,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained(model_path)

C:\Users\riede\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\riede\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- configuration_RW.py
.


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin C:\Users\riede\AppData\Roaming\Python\Python311\site-packages\bitsandbytes\libbitsandbytes_cpu.so
False
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: WARNING! libcuda.so not found! Do you have a CUDA driver installed? If you are on a cluster, make sure you are on a CUDA machine!
CUDA SETUP: Loading binary C:\Users\riede\AppData\Roaming\Python\Python311\site-packages\bitsandbytes\libbitsandbytes_cpu.so...
argument of type 'WindowsPath' is not iterable
CUDA SETUP: Problem: The main issue seems to be that the main CUDA library was not detected.
CUDA SETUP: Solution 1): Your paths are probably not up-to-date. You can update them via: sudo ldconfig.
CU

C:\Users\riede\AppData\Roaming\Python\Python311\site-packages\bitsandbytes\cuda_setup\main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {WindowsPath('vs/workbench/api/node/extensionHostProcess')}
  warn(msg)
C:\Users\riede\AppData\Roaming\Python\Python311\site-packages\bitsandbytes\cuda_setup\main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {WindowsPath('module'), WindowsPath('/matplotlib_inline.backend_inline')}
  warn(msg)
C:\Users\riede\AppData\Roaming\Python\Python311\site-packages\bitsandbytes\cuda_setup\main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {WindowsPath('/usr/local/cuda/lib64')}
  warn(msg)
C:\Users\riede\AppData\Roaming\Python\Python311\site-packages\bitsandbytes\cuda_setup\main.py:149: UserWarning: WARNING: No libcudart.so found! Install CUDA or the cudatoolkit package (anaconda)!
 

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│    1 model_path = "tiiuae/falcon-7b-instruct"                                                    │
│    2 config = AutoConfig.from_pretrained(model_path, trust_remote_code=True)                     │
│ ❱  3 model = AutoModelForCausalLM.from_pretrained(                                               │
│    4 │   model_path,                                                                             │
│    5 │   config=config,                                                                          │
│    6 │   trust_remote_code=True,                                                                 │
│                                                                                                  │
│ C:\Users\riede\AppData\Roaming\Python\Python311\site-packages\transformers\models\auto\auto_fact │
│ ory.py:479 in from_pretrained                                                                    │
│                                                                                                  │
│   476 │   │   │   │   class_ref, pretrained_model_name_or_path, **hub_kwargs, **kwargs           │
│   477 │   │   │   )                                                                              │
│   478 │   │   │   _ = hub_kwargs.pop("code_revision", None)                                      │
│ ❱ 479 │   │   │   return model_class.from_pretrained(                                            │
│   480 │   │   │   │   pretrained_model_name_or_path, *model_args, config=config, **hub_kwargs,   │
│   481 │   │   │   )                                                                              │
│   482 │   │   elif type(config) in cls._model_mapping.keys():                                    │
│                                                                                                  │
│ C:\Users\riede\AppData\Roaming\Python\Python311\site-packages\transformers\modeling_utils.py:268 │
│ 5 in from_pretrained                                                                             │
│                                                                                                  │
│   2682 │   │   │   keep_in_fp32_modules = []                                                     │
│   2683 │   │                                                                                     │
│   2684 │   │   if load_in_8bit or load_in_4bit:                                                  │
│ ❱ 2685 │   │   │   from .utils.bitsandbytes import get_keys_to_not_convert, replace_with_bnb_li  │
│   2686 │   │   │                                                                                 │
│   2687 │   │   │   llm_int8_skip_modules = quantization_config.llm_int8_skip_modules             │
│   2688 │   │   │   load_in_8bit_fp32_cpu_offload = quantization_config.llm_int8_enable_fp32_cpu  │
│                                                                                                  │
│ C:\Users\riede\AppData\Roaming\Python\Python311\site-packages\transformers\utils\bitsandbytes.py │
│ :11 in <module>                                                                                  │
│                                                                                                  │
│     8                                                                                            │
│     9                                                                                            │
│    10 if is_bitsandbytes_available():                                                            │
│ ❱  11 │   import bitsandbytes as bnb                                                             │
│    12 │   import torch                                     

In [ ]:
model.save_pretrained("falcon_7b_4bit", from_pt=True) 

In [ ]:
prompts = [
        "Act as a fitness trainer and health expert. You should create 1 short tipp why the user should be active to improve his health. Return the 1 tipp and nothing else.",
        "Act as a fitness trainer and health expert. You should create 1 short tipp why the user should be active to improve his health.",
        "Write two short activities that helps the user to stay healthy.",
        "Act as a fitness trainer and health expert. You should create 1 short tipp why the user should be active to improve his health. Return the 1 tipp and nothing else.",
        "Act as a fitness trainer and health expert. You should create 1 short tipp why the user should be active to improve his health.",
        "Write two short activities that helps the user to stay healthy.",
        "Act as a fitness trainer and health expert. You should create 1 short tipp why the user should be active to improve his health. Return the 1 tipp and nothing else.",
        "Act as a fitness trainer and health expert. You should create 1 short tipp why the user should be active to improve his health.",
        "Write two short activities that helps the user to stay healthy."
    ]
mem_consumptions = []
execution_times = []

for i in range(len(prompts)):
    mem_consumption, exec_time, outputs = measure_perf(prompts[i], model, tokenizer)
    mem_consumptions.append(mem_consumption)
    execution_times.append(exec_time)
    print(tokenizer.decode(outputs[0]))

plot_results(mem_consumptions, execution_times)